In [9]:
from langchain_community.document_loaders import PyPDFLoader,DirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [7]:
def load_data(data_dir_path): 
    loader = DirectoryLoader(data_dir_path, glob="*.pdf", loader_cls = PyPDFLoader)
    documents = loader.load()
    return documents

In [8]:
extracted_data = load_data("data/")

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7accb91a2d40>>
Traceback (most recent call last):
  File "/home/rahul/miniconda3/envs/chatbot/lib/python3.10/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(
KeyboardInterrupt: 


In [ ]:
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks = text_splitter.split_documents(extracted_data)
    return text_chunks

In [ ]:
text_chunks = text_split(extracted_data)
print("Length of text_chunks: ", len(text_chunks))

Length of text_chunks:  7023


In [10]:
from langchain.embeddings import HuggingFaceEmbeddings

In [11]:
embeddings = HuggingFaceEmbeddings(model_name = "all-MiniLM-L6-v2")

/tmp/ipykernel_45402/4186238865.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name = "all-MiniLM-L6-v2")
/home/rahul/miniconda3/envs/chatbot/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
query_result = embeddings.aembed_query("Hey my name is Rahul")

In [15]:
from dotenv import load_dotenv
load_dotenv()
import os

In [16]:
PINECONE_API_KEY = os.environ.get("PINECONE_API_KEY")

In [17]:
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key=PINECONE_API_KEY)

In [22]:
index_name = "medical-chatbot"

# pc.create_index(
#     name=index_name,
#     dimension=384,
#     metric="cosine",
#     spec=ServerlessSpec(
#         cloud="aws",
#         region="us-east-1"
#     ) 
# )

In [19]:
import os

os.environ['PINECONE_API_KEY'] = PINECONE_API_KEY

In [20]:
from langchain_pinecone import PineconeVectorStore

In [ ]:
# docsearch = PineconeVectorStore.from_documents(
#     documents=text_chunks,
#     index_name=index_name,
#     embedding=embeddings
#     )

In [23]:
get_op = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
    )

In [24]:
get_op

In [25]:
retriever = get_op.as_retriever(search_type="similarity", search_kwards={"k":3})

In [35]:
retriever_result = retriever.invoke("Acne")
print("Documents returned by retriever:")
for doc in retriever_result:
    print(doc.page_content)

Documents returned by retriever:
thing that irritates the skin and is manifested by one or
more lines of red, swollen, blistered skin that may itch or
GALE ENCYCLOPEDIA OF MEDICINE 21036
Dermatitis
in contact with an irritating substance. The symptoms
can take many forms: redness, itching , crusting,
swelling, blistering, oozing, dryness, scaliness, thicken-
ing of the skin, and a feeling of warmth at the site of con-
tact. In extreme cases, severe blistering can occur and
open sores can form. Jobs that require frequent skin
exposure to water, such as hairdressing and food prepara-
tion, can make the skin more susceptible to ICD.
Allergic contact dermatitis (ACD) results when
Corticosteriod —A group of synthetic hormones
that are used to prevent or reduce inflammation.
Toxic effects may result from rapid withdrawal after
prolonged use or from continued use of large doses.
Patch test—A skin test that is done to identify aller-
gens. A suspected substance is applied to the skin.
After 24

In [27]:
from langchain.llms import Together
load_dotenv()

api_key = os.getenv("TOGETHER_API_KEY")

In [28]:
llm = Together(
    model="meta-llama/Llama-3-70b-chat-hf"
)

/tmp/ipykernel_45402/3910092519.py:1: LangChainDeprecationWarning: The class `Together` was deprecated in LangChain 0.0.12 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-together package and should be used instead. To use it run `pip install -U :class:`~langchain-together` and import as `from :class:`~langchain_together import Together``.
  llm = Together(


In [39]:
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you don't know. "
    "Use three sentences maximum and keep the answer concise.\n\n{context}"
)

In [41]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [42]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain

question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [44]:
user_question = "What is acne?"

# Invoke the RAG chain, passing the user's question
response = rag_chain.invoke({"input": user_question})

# Print the result
print("Response: ",response)

print("Answer:", response["answer"])

Response:  {'input': 'What is acne?', 'context': [Document(id='94c77c22-5675-40d6-8b9a-22d657f6c28b', metadata={'creationdate': '2004-12-18T17:16:32-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:35:04-06:00', 'page': 425.0, 'page_label': '426', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'data/Gale Encyclopedia of Medicine. Vol. 2. 2nd ed.pdf', 'total_pages': 759.0}, page_content='Corticosteriod —A group of synthetic hormones\nthat are used to prevent or reduce inflammation.\nToxic effects may result from rapid withdrawal after\nprolonged use or from continued use of large doses.\nPatch test—A skin test that is done to identify aller-\ngens. A suspected substance is applied to the skin.\nAfter 24–48 hours, if the area is red and swollen,\nthe test is positive for that substance. If no reaction\noccurs, another substance is applied. This is con-'), Document(id='5e904193-ca56-4cf2-b45a-adf4d0908de6', metadata={'creationdate': '2004-12-18T17:16:32-05:00', 'creator': 'PyPDF',

In [47]:
context_text = "\n".join([doc.page_content for doc in retriever_result])
prompt_text = system_prompt.format(context=context_text, input="What is diabetes?")
response = llm.invoke(prompt_text)
print("LLM Response:", response)


LLM Response: 
tact dermatitis occurs when the skin comes into con-
tact with an irritating substance.
